# Course Scheduling - Complete PuLP Implementation
## All 7 Hard Constraints Implemented

This notebook implements a complete course scheduling optimization model with all mathematical constraints.

## 1. Import Required Libraries

In [14]:
import json
import pulp
from pprint import pprint

## 2. Load All Parameters

Run the parameter loader script to get all data organized.

In [15]:
# Run the parameter loading script
%run load_parameters.py

Programs (P): ['CS_Y1', 'CS_Y2', 'DS_Y1', 'DS_Y2']
Courses (C): ['BCS1220', 'BCS1440', 'BCS1530', 'BCS2110', 'BCS2210', 'BCS2720', 'KEN1440', 'KEN1530', 'KEN1220', 'KEN2230', 'KEN2240', 'KEN2530']
Rooms (R): ['EDP150', 'C0.004', 'C0.008', 'C0.016', 'C0.020', 'C1.005', 'C1.015', 'C2.007']
Lecturers (L): ['Tom', 'Ottie', 'Filippe', 'Tony', 'Francessco', 'Tang', 'Marieke', 'Stefan', 'Yan', 'Ashish 10', 'Charis']

Student counts (n_p): {'CS_Y1': 220, 'CS_Y2': 165, 'DS_Y1': 180, 'DS_Y2': 218}

Courses per program (Courses_p): {'CS_Y1': ['BCS1220', 'BCS1440', 'BCS1530'], 'CS_Y2': ['BCS2110', 'BCS2210', 'BCS2720'], 'DS_Y1': ['KEN1220', 'KEN1440', 'KEN1530'], 'DS_Y2': ['KEN2230', 'KEN2240', 'KEN2530']}

Course timelines (Timeline_c): {'BCS1220': ['C', 'C', 'T'], 'BCS1440': ['C', 'C', 'T'], 'BCS1530': ['C', 'C', 'T'], 'BCS2110': ['C', 'C', 'T'], 'BCS2210': ['C', 'C', 'T'], 'BCS2720': ['C', 'C', 'T'], 'KEN1440': ['C', 'C', 'T'], 'KEN1530': ['C', 'C', 'T'], 'KEN1220': ['C', 'C', 'T'], 'KEN2230': 

## 3. Create Decision Variables

Create binary variables x[c,i,r,d,t] for all possible scheduling combinations.

In [16]:
# Create the optimization model
model = pulp.LpProblem("Course_Scheduling", pulp.LpMinimize)

# Create decision variables
x = pulp.LpVariable.dicts(
    "schedule",
    [(c, i, r, d, t)
     for c in params['C']
     for i in params['I'][c]
     for r in params['R']
     for d in params['D']
     for t in params['T']],
    cat='Binary'
)

print(f"Created {len(x)} binary decision variables")
print(f"This represents all possible scheduling combinations!")

Created 5760 binary decision variables
This represents all possible scheduling combinations!


## 4. Set Objective Function

For MVP, we just want feasibility (any valid schedule), so we minimize 0.

In [17]:
# Feasibility only - minimize 0 (dummy objective)
model += 0, "Feasibility_Only"

## 5. Add All Constraints

Now we add all 7 hard constraints to the model.

### Constraint 1: Each session scheduled exactly once

In [18]:
# ∀c ∈ C, ∀i ∈ Ic: ∑(r∈R, d∈D, t∈T) x[c,i,r,d,t] = 1

constraint_count = 0
for c in params['C']:
    for i in params['I'][c]:
        model += (
            pulp.lpSum(
                x[c, i, r, d, t]
                for r in params['R']
                for d in params['D']
                for t in params['T']
            ) == 1,
            f"C1_Session_{c}_i{i}_scheduled_once"
        )
        constraint_count += 1

print(f"✓ Constraint 1: {constraint_count} constraints added")

✓ Constraint 1: 36 constraints added


### Constraint 2: Room capacity ≥ 50% of students

In [19]:
# ∀c ∈ C, ∀i ∈ Ic: If x[c,i,r,d,t] = 1, then Cap[r] ≥ 0.5 × students[c]

constraint_count = 0
students_per_course = params['students_per_course']

for c in params['C']:
    min_capacity = 0.5 * students_per_course[c]
    
    for i in params['I'][c]:
        for r in params['R']:
            if params['Cap'][r] >= min_capacity:
                # Room is big enough - add capacity constraint
                for d in params['D']:
                    for t in params['T']:
                        model += (
                            x[c, i, r, d, t] * students_per_course[c] <= 2 * params['Cap'][r],
                            f"C2_Capacity_{c}_i{i}_r{r}_d{d}_t{t}"
                        )
                        constraint_count += 1
            else:
                # Room too small - prevent scheduling
                for d in params['D']:
                    for t in params['T']:
                        model += (
                            x[c, i, r, d, t] == 0,
                            f"C2_TooSmall_{c}_i{i}_r{r}_d{d}_t{t}"
                        )
                        constraint_count += 1

print(f"✓ Constraint 2: {constraint_count} constraints added")

✓ Constraint 2: 5760 constraints added


### Constraint 3: No room double-booking

In [20]:
# ∀r ∈ R, ∀d ∈ D, ∀t ∈ T: ∑(c∈C, i∈Ic) x[c,i,r,d,t] ≤ 1

constraint_count = 0
for r in params['R']:
    for d in params['D']:
        for t in params['T']:
            model += (
                pulp.lpSum(
                    x[c, i, r, d, t]
                    for c in params['C']
                    for i in params['I'][c]
                ) <= 1,
                f"C3_NoDoubleBook_r{r}_d{d}_t{t}"
            )
            constraint_count += 1

print(f"✓ Constraint 3: {constraint_count} constraints added")

✓ Constraint 3: 160 constraints added


### Constraint 4: Lecturer unavailability

In [21]:
# ∀c ∈ C, ∀i ∈ Ic, ∀d ∈ D, ∀t ∈ T:
# If U[Lecturer[c], d] = 1, then ∑(r∈R) x[c,i,r,d,t] = 0

DAY_NAMES = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday'}
constraint_count = 0

for c in params['C']:
    lecturer = params['Lecturer'][c]
    unavailable_days = params['U'].get(lecturer, [])
    
    for i in params['I'][c]:
        for d in params['D']:
            if DAY_NAMES[d] in unavailable_days:
                for t in params['T']:
                    model += (
                        pulp.lpSum(
                            x[c, i, r, d, t]
                            for r in params['R']
                        ) == 0,
                        f"C4_Unavailable_{c}_i{i}_{lecturer}_d{d}_t{t}"
                    )
                    constraint_count += 1

print(f"✓ Constraint 4: {constraint_count} constraints added")

✓ Constraint 4: 0 constraints added


### Constraint 5: Lecturer no double-booking

In [9]:
# ∀l ∈ L, ∀d ∈ D, ∀t ∈ T: ∑(c: Lecturer[c]=l, i∈Ic, r∈R) x[c,i,r,d,t] ≤ 1

constraint_count = 0
for l in params['L']:
    courses_by_lecturer = [c for c in params['C'] if params['Lecturer'][c] == l]
    
    for d in params['D']:
        for t in params['T']:
            model += (
                pulp.lpSum(
                    x[c, i, r, d, t]
                    for c in courses_by_lecturer
                    for i in params['I'][c]
                    for r in params['R']
                ) <= 1,
                f"C5_NoDoubleTeach_{l}_d{d}_t{t}"
            )
            constraint_count += 1

print(f"✓ Constraint 5: {constraint_count} constraints added")

✓ Constraint 5: 220 constraints added


### Constraint 6: Student no overlap

In [10]:
# ∀p ∈ P, ∀d ∈ D, ∀t ∈ T: ∑(c∈Courses[p], i∈Ic, r∈R) x[c,i,r,d,t] ≤ 1

constraint_count = 0
for p in params['P']:
    courses_in_program = params['Courses'][p]
    
    for d in params['D']:
        for t in params['T']:
            model += (
                pulp.lpSum(
                    x[c, i, r, d, t]
                    for c in courses_in_program
                    for i in params['I'][c]
                    for r in params['R']
                ) <= 1,
                f"C6_NoStudentOverlap_{p}_d{d}_t{t}"
            )
            constraint_count += 1

print(f"✓ Constraint 6: {constraint_count} constraints added")

✓ Constraint 6: 80 constraints added


### Constraint 7: Chronological ordering of sessions

In [11]:
# ∀c ∈ C, ∀i ∈ {0,...,|Ic|-2}:
# ∑(d,t,r) (5×d + t) × x[c,i,r,d,t] < ∑(d,t,r) (5×d + t) × x[c,i+1,r,d,t]

constraint_count = 0
for c in params['C']:
    for i in range(len(params['I'][c]) - 1):
        time_session_i = pulp.lpSum(
            (5 * d + t) * x[c, i, r, d, t]
            for r in params['R']
            for d in params['D']
            for t in params['T']
        )
        
        time_session_i_plus_1 = pulp.lpSum(
            (5 * d + t) * x[c, i + 1, r, d, t]
            for r in params['R']
            for d in params['D']
            for t in params['T']
        )
        
        model += (
            time_session_i + 1 <= time_session_i_plus_1,
            f"C7_Ordering_{c}_i{i}_before_i{i+1}"
        )
        constraint_count += 1

print(f"✓ Constraint 7: {constraint_count} constraints added")

✓ Constraint 7: 24 constraints added


## 6. Solve the Model

In [13]:
# Solve the optimization problem
print("\nSolving the optimization problem...")
print("This may take a few minutes...\n")

status = model.solve()

print("="*80)
print(f"Solution Status: {pulp.LpStatus[status]}")
print("="*80)

if status == pulp.LpStatusOptimal:
    print("\n✅ SUCCESS! Found a feasible schedule!")
elif status == pulp.LpStatusInfeasible:
    print("\n❌ INFEASIBLE: No valid schedule exists with current constraints")
else:
    print(f"\n⚠️  Solver returned status: {pulp.LpStatus[status]}")


Solving the optimization problem...
This may take a few minutes...

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/lib/python3.13/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/8m/_hd_3f8d7gd1yprsfd9_9fy80000gn/T/99fdd230f3ad4c5a9edbe722496f156f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8m/_hd_3f8d7gd1yprsfd9_9fy80000gn/T/99fdd230f3ad4c5a9edbe722496f156f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6285 COLUMNS
At line 51407 RHS
At line 57688 BOUNDS
At line 63450 ENDATA
Problem MODEL has 6280 rows, 5761 columns and 33600 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.03 seconds
Cgl0002I 2460 variables fixed
Cgl0000I Cut generators found to be infeasible! (or unbounded)
Pre-processing says infeasible or unbounded
Option for printingOptions changed from nor

## 7. Extract and Display the Schedule

In [ ]:
if status == pulp.LpStatusOptimal:
    DAY_NAMES = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday'}
    TIME_NAMES = {1: '08:30-10:30', 2: '11:00-13:00', 3: '13:30-15:30', 4: '16:00-18:00'}
    
    # Extract scheduled sessions
    schedule = []
    for (c, i, r, d, t), var in x.items():
        if var.varValue == 1:
            schedule.append({
                'Course': c,
                'Session': i,
                'Type': params['Timeline'][c][i],
                'Room': r,
                'Day': DAY_NAMES[d],
                'Time': TIME_NAMES[t],
                'Lecturer': params['Lecturer'][c],
                'Students': params['students_per_course'][c]
            })
    
    # Sort by day and time
    day_order = {v: k for k, v in DAY_NAMES.items()}
    time_order = {v: k for k, v in TIME_NAMES.items()}
    
    schedule.sort(key=lambda x: (day_order[x['Day']], time_order[x['Time']]))
    
    # Display schedule
    print("\n" + "="*100)
    print("FINAL SCHEDULE")
    print("="*100)
    
    current_day = None
    for session in schedule:
        if session['Day'] != current_day:
            current_day = session['Day']
            print(f"\n{'='*100}")
            print(f"{current_day.upper()}")
            print(f"{'='*100}")
        
        print(f"{session['Time']:15} | {session['Course']:8} Session {session['Session']} ({session['Type']}) | "
              f"Room {session['Room']:7} | {session['Lecturer']:12} | {session['Students']} students")
    
    print("\n" + "="*100)
    print(f"Total sessions scheduled: {len(schedule)}")
    print("="*100)

## 8. Verify Constraint Satisfaction (Optional)

In [ ]:
if status == pulp.LpStatusOptimal:
    print("\nVerifying constraint satisfaction...\n")
    
    # Check each session is scheduled once
    sessions_scheduled = {}
    for c in params['C']:
        for i in params['I'][c]:
            count = sum(x[c, i, r, d, t].varValue for r in params['R'] 
                       for d in params['D'] for t in params['T'])
            sessions_scheduled[(c, i)] = count
    
    all_scheduled_once = all(count == 1 for count in sessions_scheduled.values())
    print(f"✓ All sessions scheduled exactly once: {all_scheduled_once}")
    
    # More verification can be added here
    print("\n✅ All basic verifications passed!")